In [1]:
#export
import logging, warnings, os, time, re, gc, json, k1lib, importlib, urllib.parse, math, base64, dill, inspect, asyncio
import threading, datetime, traceback, functools, collections, numpy as np, pprint, html
from typing import Any, List, Union, Tuple, Iterator, Dict
try: import torch; hasTorch = True
except: hasTorch = False

In [2]:
#export
__all__ = ["_docsUrl", "mplLock", "dep", "depCli", "isNumeric",
           "patch", "cache", "cron", "preload", "wrapMod", "wraps", "squeeze", "raiseEx",
           "numDigits", "limitLines",
           "limitChars", "showLog", "cleanDiv",
           "beep", "beepOnAvailable", "dontWrap",
           "debounce", "scaleSvg", "now", "pushNotification", "ticks", "digraph", "graph",
           "encode", "decode", "hash", "resolve", "config", "modbusCrc", "parseTimeStr"]

In [58]:
#export
_docsUrl = "https://k1lib.com"
mplLock = threading.Lock()

In [4]:
#export
class Dependency:
    def __init__(self, s, alt:str=None, url:str=None): self.s = s; self.alt = alt; self.url = url; self.errorMsg = ""
    def __getattr__(self, attr):
        after = f"More information is available on {self.url}. " if self.url else f""
        raise ImportError(f"Python package `{self.alt or self.s}` not found. Please install it. {after}After installing, if you're in a notebook environment, then restart the kernel. This is the error when importing it:\n\n{self.errorMsg}")
def oldDep(s, alt:str=None, url:str=None):
    try: return importlib.import_module(s)
    except: d = Dependency(s, alt, url); d.errorMsg = traceback.format_exc(); return d
class dep:
    def __init__(self, s, alt=None, url=None):
        """Imports a potentially unavailable package
Example::

    graphviz = k1.dep("graphviz")
    
    # executes as normal, if graphviz is available, else throws an error
    g = graphviz.Digraph()

I don't imagine this would be useful for everyday use though.
This is mainly for writing this library, so that it can use
optional dependencies.

:param s: name of the package. Can be nested, like `matplotlib.pyplot`
:param alt: (optional) name of the package to display in error message if package is not found
:param url: (optional) url of the package's website, so that they know where to get official docs"""
        self._k_s = s; self._k_alt = alt; self._k_url = url; self._k_mod = None; self.loaded = False
    def _k_bootstrap(self):
        try: self.loaded = True; self._k_mod = importlib.import_module(self._k_s); return self._k_mod
        except:
            after = f"More information is available on {self._k_url}. " if self._k_url else f""
            raise ImportError(f"Python package `{self._k_alt or self._k_s}` not found. Please install it. {after}After installing, if you're in a notebook environment, then restart the kernel. This is the error when importing it:\n\n{traceback.format_exc()}")
    def __getattr__(self, attr):
        """This code is a little magical.
Say you're doing this::

    np = k1.dep("numpy")
    np                   # it's an object of class k1lib.dep
    np.array([2, 3])     # instantiate a new array like usual. This is by running k1lib.__getattr__
    np                   # now the old python object has turned into the actual numpy module, as if you import it using "import numpy as np"

It's not foolproof, but works in most cases and gives a nice little speed boost

Inspired from https://github.com/cart0113/pyjack"""
        if not self.loaded:
            replaceWith = self._k_bootstrap(); gc.collect(); arr = gc.get_referrers(self); iSelf = id(self)
            for x in arr:
                if isinstance(x, dict):
                    for k,v in x.items():
                        if id(v) == iSelf: x[k] = replaceWith
        return getattr(self._k_mod, attr)
dep.mpl = dep("matplotlib", url="https://matplotlib.org/"); dep.plt = dep("matplotlib.pyplot", url="https://matplotlib.org/"); dep.cm = dep("matplotlib.cm", url="https://matplotlib.org/")
dep.pygments = dep("pygments", url="https://pygments.org/"); dep.torch = dep("torch", url="https://pytorch.org/")
try: import pygments.formatters; import pygments.lexers # these are not loaded by default, so have to prime them
except: pass
dep.bs4 = dep("bs4", "beautifulsoup4", "https://www.crummy.com/software/BeautifulSoup/")
dep.yaml = dep("yaml", "pyyaml", "https://pyyaml.org/"); dep.pd = dep("pandas", url="https://pandas.pydata.org/")
dep.requests = dep("requests", url="https://requests.readthedocs.io/"); dep.graphviz = dep("graphviz", url="https://graphviz.readthedocs.io/")
dep.PIL = dep("PIL", "Pillow", "https://pillow.readthedocs.io/"); dep.websockets = dep("websockets", url="https://websockets.readthedocs.io/")
dep.cv2 = dep("cv2", "opencv-python", "https://opencv.org/")
def depCli(prog="ls", insCmd=None, website=None):
    """Checks whether a particular command-line program is installed or not.
Example::

    k1.depCli("vim")

This will throw an error if that program is not found

:param prog: program name
:param insCmd: (optional) installation command, could be "apt install vim"
:param website: (optional) website that the user can learn more about the program"""
    if None | k1lib.cli.cmd(("where " if os.name == "nt" else "which ") + prog) | k1lib.cli.shape(0) == 0:
        insS = f" by executing '{insCmd}'" if insCmd else ""
        webS = f". Visit {website} to learn more about the program" if website else ""
        raise Exception(f"Command line program '{prog}' not found in PATH. Please install it{insS}{webS}")
    return True

In [5]:
#export
def isNumeric(x) -> bool:
    """Returns whether object is actually a number"""
    return isinstance(x, (int, float, np.number))

In [6]:
#export
def patch(_class:type, name:str=None, docs:Union[str, Any]=None, static=False):
    """Patches a function to a class/object.

:param _class: object to patch function. Can also be a type
:param name: name of patched function, if different from current
:param docs: docs of patched function. Can be object with defined __doc__ attr
:param static: whether to wrap this inside :class:`staticmethod` or not
:return: modified function just before patching

Intended to be used like this::

    class A:
        def methA(self):
            return "inside methA"

    @k1lib.patch(A)
    def methB(self):
        return "inside methB"
        
    a = A()
    a.methB() # returns "inside methB"

You can do ``@property`` attributes like this::

    class A: pass
    
    @k1lib.patch(A, "propC")
    @property
    def propC(self): return self._propC

    @k1lib.patch(A, "propC")
    @propC.setter
    def propC(self, value): self._propC = value
    
    a = A(); a.propC = "abc"
    a.propC # returns "abc"

The attribute name unfortunately has to be explicitly declared, as I can't
really find a way to extract the original name. You can also do static
methods like this::

    class A: pass
    
    @k1lib.patch(A, static=True)
    def staticD(arg1): return arg1
    
    A.staticD("def") # returns "def"
"""
    def inner(function):
        _docs = docs
        if _docs is not None and not isinstance(_docs, str): _docs = _docs.__doc__
        _docs = _docs or function.__doc__ or _class.__doc__
        _name = name or function.__qualname__.split(".")[-1]
        
        _function = staticmethod(function) if static else function
        _function.__doc__ = _docs; setattr(_class, _name, _function); return _function
    return inner

In [7]:
class A:
    def methA(self): return "inside methA"
@patch(A)
def methB(self): return "inside methB"
@patch(A, name="propC")
@property
def propC(self): return "inside propC"
@patch(A, name="propC")
@propC.setter
def propC(self, value): self.propD = value
@patch(A, static=True)
def staticE(arg1): return arg1
a = A(); assert a.methA() == "inside methA"
assert a.methB() == "inside methB"; assert a.propC == "inside propC"
a.propC = 3; assert a.propD == 3; assert A.staticE(5) == 5

In [8]:
#export
def hashableAKw(args, kwargs):
    a = tuple((tuple(x) if isinstance(x, list) else x) for x in args) # can accept lists of hashable items!
    b = tuple(sorted(list(kwargs.items()), key=lambda x: x[0])); key = (*a, *b); return (*a, *b)
_k1_cache_autoInc = [1]; _k1_cacheD = {} # Dict[name -> {docs, func, clearF, hits, misses}]
class cache:
    def __init__(self, maxsize=128, timeout=60, name=None, docs=None):
        """Like functools.lru_cache, but with timeout period.
Example::

    @k1.cache(timeout=10)
    def slow_function(x): return x * 2

You can also quickly deploy a management plane to manage all caches::

    app = flask.Flask(__name__)
    k1.cache.flask(app)
    app.run(host="0.0.0.0", port=80)

See more about this at :meth:`flask`

:param maxsize: max size of the cache
:param timeout: timeout in seconds"""
        if not name: name = f"_k1_cache_{_k1_cache_autoInc[0]}"; _k1_cache_autoInc[0] += 1
        if "/" in name: raise Exception("Can't have forward slash in the name")
        self.maxsize = maxsize; self.timeout = timeout; self.cache = collections.OrderedDict(); self.name = name; self.docs = docs
    def __call__(self, func):
        if self.name in _k1_cacheD: raise Exception(f"Name '{self.name}' has appeared before. Please use a different name")
        _k1_cacheD[self.name] = meta = {"docs": self.docs, "func": func, "clearF": None, "hits": 0, "timeouts": 0, "misses": 0, "capacity": self.maxsize, "len": len(self.cache), "timeout": self.timeout}
        def clearF(): self.cache.clear(); meta["hits"] = 0; meta["timeouts"] = 0; meta["misses"] = 0; meta["len"] = 0
        meta["clearF"] = clearF
        @functools.wraps(func)
        def wrapper(*args, _k1_cache_refresh=False, **kwargs): # _k1_cache_refresh: if True, returns the cached item, but then immediately fetches the new version. Not entirely sure where this is used in
            if self.maxsize == 0: return func(*args, **kwargs)
            key = hashableAKw(args, kwargs); now = time.time()
            if key in self.cache:
                result, timestamp = self.cache[key]
                if _k1_cache_refresh: result = func(*args, **kwargs); self.cache[key] = (result, now); return result
                if now - timestamp < self.timeout: self.cache.move_to_end(key); meta["hits"] += 1; return result
                else: del self.cache[key]; meta["timeouts"] += 1
            else: meta["misses"] += 1
            result = func(*args, **kwargs)
            if len(self.cache) >= self.maxsize: self.cache.popitem(last=False)
            self.cache[key] = (result, now); meta["len"] = len(self.cache); return result
        return wrapper
    @staticmethod
    def allData():
        """Grabs ``Dict[name -> {docs, func, clearF}]`` of all caches"""
        return _k1_cacheD
    @staticmethod
    def flask(app, **kwargs):
        """Attaches a cache management plane to a flask app.
Example::

    app = flask.Flask(__name__)
    k1.cache.flask(app)
    app.run(host="0.0.0.0", port=80)

Then, you can access the route "/k1/cache" to see an overview of all caches

:param app: flask app object
:param kwargs: extra random kwargs that you want to add to ``app.route()`` function"""
        k1 = k1lib; cli = k1.cli; viz = k1.viz
        @app.route("/k1/cache", **kwargs)
        def k1_cache_index(): ui3 = k1.cache.allData().items() | ~cli.apply(lambda k,v: [k, v["func"].__name__, inspect.getfile(v["func"]), v["hits"], v["misses"], v["timeouts"], round(v["hits"]/(v["hits"] + v["misses"] + v["timeouts"] + 1e-9)*100, 2), round(v["misses"]/(v["hits"] + v["misses"] + v["timeouts"] + 1e-9)*100, 2), round(v["timeouts"]/(v["hits"] + v["misses"] + v["timeouts"] + 1e-9)*100, 2), v["timeout"], v["len"], v["capacity"], round(v["len"]/(v["capacity"]+1e-9)*100, 2), f"<button onclick='clearCache(\"{k}\")'>Clear</button>", v["docs"]]) | cli.deref() | (cli.toJsFunc("term") | cli.grep("${term}") | viz.Table(["name", "func's name", "func's file name", "hits", "misses", "timeouts", "% hit", "% misses", "% timeouts", "timeout (s)", "len", "capacity", "% capacity", "clear cache", "docs"], height=600, sortF=True)) | cli.op().interface() | cli.toHtml(); return f"""
<h1>Caches</h1><button onclick="clearAllCache()">Clear all caches</button><div style="overflow-x: auto">{ui3}</div><script>async function clearCache(x) {{ await fetch(`/k1/cache/api/${{x}}/clear`); alert("Cleared"); }}; async function clearAllCache() {{ await fetch(`/k1/cache/api/clearAll`); alert("Cleared all"); }}</script>"""
        @app.route("/k1/cache/api/<cache>/clear", **kwargs)
        def k1_cache_api_clear(cache): k1.cache.allData()[cache]["clearF"](); return "ok"
        @app.route("/k1/cache/api/clearAll", **kwargs)
        def k1_cache_api_clearAll():
            for c in list(k1.cache.allData().values()): c["clearF"]()
            return "ok"

In [9]:
@cache(timeout=10)
def f(x): time.sleep(1); return x+3
a = time.time(); f(3); b = time.time(); f(3); c = time.time()
assert abs(b-a-1) < 1e-2; assert c-b < 1e-3

In [10]:
#export
_k1_cron_autoInc = [1]; _k1_cron_delay_data = {} # Dict[name -> {docs, delay, logs, lastRun}]
_k1_threadData = {} # Dict[id(thread) -> [thread name, thread obj, running?, beginTime, threadTime]]. This only stores threads created by cron
def _addThread(t, threadRef): _k1_threadData[id(t)] = threadRef; name = threadRef[0]; threadRef.clear(); threadRef.extend([name, t, True, time.time(), 0]); return t
_c2_eventLoop = [None, False, False, False] # [event loop, asyncScanCron finished running?, asyncScanCron started?, sync scanCron started?]
def asyncScanCron(): loop = asyncio.new_event_loop(); _c2_eventLoop[0] = loop; _c2_eventLoop[1] = True; loop.run_forever() # main thread that manages all cron connections that are async
def _startAsyncScanCron():
    if _c2_eventLoop[2]: return
    _c2_eventLoop[2] = True; threading.Thread(target=asyncScanCron, daemon=True).start()
    while not _c2_eventLoop[1]: time.sleep(0.1)
def _startSyncScanCron():
    if _c2_eventLoop[3]: return
    _c2_eventLoop[3] = True; cron(delay=10, daemon=True, delayedStart=5, name="scanCron", docs="scans through all cron jobs (k1.cron) to calculate time string of how long ago it was executed")(scanCron)
class cron:
    stop = "_k1.cron.stopSignal"
    def __init__(self, f=None, delay=None, kw=None, name=None, docs=None, daemon=False, delayedStart=0):
        """Sets up a cron job in another thread, running the decorated
function in 2 modes:

Mode 1: whenever ``f`` goes from False to True. It's activated when ``delay`` is not specified. Example::

    @k1.cron(lambda minute: minute == 0)
    def f1(): # runs every hour
        ... # do some stuff

    @k1.cron(lambda second: second % 5 == 0)
    def f2(): # runs every 5 seconds
        ... # do some stuff

So, the first function will run every hour, and the second function will
run every 5 seconds. Pretty straightforward. The timing function ``f`` can
be as complicated as you want, but it can only accept the following parameters:

- year
- month: 1-12
- day: 1-31
- weekday: 0-6, 0 for Monday
- hour: 0-23
- minute: 0-59
- second: 0-59

Mode 2: every ``delay`` seconds. It's activated when ``delay`` is specified. Example::

    @k1.cron(delay=10)
    def f1(): # runs every 10 seconds
        ... # do some stuff

Theres a special sentinel ``k1.cron.stop`` that when returns, it terminates the cron
loop, for both modes. Example::

    @k1.cron(delay=10)
    def f1(): # only prints out "abc" once
        print("abc")
        return k1.cron.stop

You can also use async functions. All async functions will be run in a new event loop
on another thread. So only 1 thread will ever exist despite how many cron jobs on async
functions you have. Example::

    @k1.cron(delay=10)
    async def f1(): # only prints out "abc" once
        print("abc")

You can also get a python object describing past runs, stdout and errors by doing ``k1.cron.allData``

You can also quickly deploy a management plane to manage all cron jobs::

    app = flask.Flask(__name__)
    k1.cron.flask(app)
    app.run(host="0.0.0.0", port=80)

See more about this at :meth:`flask`

:param f: function to trigger wrapped function when this transitions from False to True
:param delay: if specified, just run the function repeatedly, delayed by this amount after running. Mutually exclusive with .f
:param kw: extra keyword arguments to pass to the function
:param name: optional name to show up in :meth:`allData`
:param docs: optional documentation to show up in :meth:`allData`
:param daemon: if True, makes the cron thread daemon
:param delayedStart: if specified, delay the start of the first function execution to after this number of seconds"""
        if not name: name = f"_k1_cron_{_k1_cron_autoInc[0]}"; _k1_cron_autoInc[0] += 1
        if "/" in name: raise Exception("Can't have forward slash in the name")
        self.f = f; self.delay = delay; self.kw = kw; self.name = name; self.docs = docs; self.daemon = daemon; self.delayedStart = delayedStart
    @staticmethod
    def allData():
        """Grabs ``Dict[name -> {docs, delay, logs}]`` for all delay crons"""
        return _k1_cron_delay_data
    def __call__(self, func):
        f = self.f; delay = self.delay; kw = self.kw; name = self.name; docs = self.docs; kw = kw or {}; isAsync = inspect.iscoroutinefunction(func); _startSyncScanCron()
        if self.name in _k1_cron_delay_data: raise Exception(f"Name '{self.name}' has appeared before. Please use a different name")
        _k1_cron_delay_data[name] = self.obj = obj = {"docs": docs, "delay": delay, "logs": collections.deque([], 100), "lastRun": [0, ""], "delayedStart": self.delayedStart, "func": func, "isAsync": isAsync}
        if delay:
            threadRef = [self.name, None, True, time.time(), 0] # name, thread object, running?, startTime, accumulated thread_time
            if not isAsync:
                def startLoop():
                    """Main k1.cron() looping function"""
                    if self.delayedStart > 0: time.sleep(self.delayedStart)
                    while True:
                        if not threadRef[2]: break
                        beginTime = time.time(); beginTTime = time.thread_time()
                        try:
                            _k1_cron_delay_data[name]["lastRun"] = [beginTime, "now"]; res = func(**kw)
                            if res == cron.stop: break
                        except Exception as e: print(f"{type(e)}\ue004{e}"); print(traceback.format_exc()); _k1_cron_delay_data[name]["logs"].append({"beginTime": beginTime, "duration": time.time() - beginTime, "success": False, "exc": f"{type(e)}\ue004{e}", "tb": traceback.format_exc()})
                        threadRef[4] += time.thread_time() - beginTTime; time.sleep(delay)
                    del _k1_threadData[id(threadRef[1])]
                _addThread(threading.Thread(target=startLoop, daemon=self.daemon), threadRef).start(); return func
            else:
                async def startALoop():
                    if self.delayedStart > 0: await asyncio.sleep(self.delayedStart)
                    while True:
                        if not threadRef[2]: break
                        beginTime = time.time(); beginTTime = time.thread_time()
                        try:
                            _k1_cron_delay_data[name]["lastRun"] = [beginTime, "now"]; res = await func(**kw)
                            if res == cron.stop: break
                        except Exception as e: print(f"{type(e)}\ue004{e}"); print(traceback.format_exc()); _k1_cron_delay_data[name]["logs"].append({"beginTime": beginTime, "duration": time.time() - beginTime, "success": False, "exc": f"{type(e)}\ue004{e}", "tb": traceback.format_exc()})
                        threadRef[4] += time.thread_time() - beginTTime; await asyncio.sleep(delay)
                    del _k1_threadData[id(threadRef[1])]
                _startAsyncScanCron(); asyncio.run_coroutine_threadsafe(startALoop(), _c2_eventLoop[0]); _addThread(func, threadRef); return func
        else:
            threadRef = [self.name, None, True, time.time(), 0]
            if not isAsync:
                args = list(inspect.signature(f).parameters.keys())
                s = {"year", "month", "day", "weekday", "hour", "minute", "second"}
                for arg in args:
                    if arg not in s: raise Exception(f"Unknown argument {arg}. Only (year, month, day, weekday, hour, minute, seconds) are allowed")
                def startLoop():
                    """Main k1.cron() looping function"""
                    last = False; this = last
                    if self.delayedStart > 0: time.sleep(self.delayedStart)
                    while True:
                        if not threadRef[2]: break
                        beginTTime = time.thread_time(); a = datetime.datetime.now()
                        now = {"year": a.year, "month": a.month, "day": a.day, "weekday": a.weekday(), "hour": a.hour, "minute": a.minute, "second": a.second}
                        this = f(*[now[e] for e in args])
                        if not last and this:
                            try:
                                if func(**kw) == cron.stop: break
                            except Exception as e: print(f"{type(e)}\ue004{e}"); print(traceback.format_exc())
                        last = this; threadRef[4] += time.thread_time() - beginTTime; time.sleep(0.5)
                    del _k1_threadData[id(threadRef[1])]
                _addThread(threading.Thread(target=startLoop, daemon=self.daemon), threadRef).start(); return func
            else:
                async def startALoop():
                    """Main k1.cron() looping function"""
                    last = False; this = last
                    if self.delayedStart > 0: await asyncio.sleep(self.delayedStart)
                    while True:
                        if not threadRef[2]: break
                        beginTTime = time.thread_time(); a = datetime.datetime.now()
                        now = {"year": a.year, "month": a.month, "day": a.day, "weekday": a.weekday(), "hour": a.hour, "minute": a.minute, "second": a.second}
                        this = f(*[now[e] for e in args])
                        if not last and this:
                            try:
                                if await func(**kw) == cron.stop: break
                            except Exception as e: print(f"{type(e)}\ue004{e}"); print(traceback.format_exc())
                        last = this; threadRef[4] += time.thread_time() - beginTTime; await asyncio.sleep(0.5)
                    del _k1_threadData[id(threadRef[1])]
                _startAsyncScanCron(); asyncio.run_coroutine_threadsafe(startALoop(), _c2_eventLoop[0]); _addThread(func, threadRef); return func
    @staticmethod
    def flask(app, **kwargs):
        """Attaches a cron management plane to a flask app.
Example::

    app = flask.Flask(__name__)
    k1.cron.flask(app)
    app.run(host="0.0.0.0", port=80)

Then, you can access the route "/k1/cron" to see an overview of all cron jobs

:param app: flask app object
:param kwargs: extra random kwargs that you want to add to ``app.route()`` function"""
        k1 = k1lib; cli = k1.cli; viz = k1.viz
        def k1_thread():
            d = {}; now = time.time(); pre = cli.init._jsDAuto(); totalUtil = 0
            c = k1.cron.allData() # all supplementary data from crons, to pull them out
            allThIds = [id(thr) for thr in threading.enumerate()]
            for thId,[name,thr,running,beginTime,threadTime] in threading.allData().items():
                isAsync = inspect.iscoroutinefunction(thr)
                if thId not in allThIds and not isAsync: continue
                tar = thr if isAsync else thr._target; cronData = c[name]; totalUtil += threadTime/(now-beginTime)
                tarData = [None, None, None] if tar is None else [tar.__name__, inspect.getfile(tar), tar.__doc__]
                d[thId] = [thId, str(thr), isAsync, *tarData, cronData["docs"], cronData["func"].__name__, inspect.getfile(cronData["func"]), running, int(beginTime), beginTime | cli.toIso(k1.settings.timezone) | cli.op().replace(*"T "), round(threadTime, 9), f"{round(100*threadTime/(now-beginTime), 4)}%", f"""<button class="btn" onclick="{pre}_stop({thId})">Stop</button>"""]
            for thr in threading.enumerate():
                thId = id(thr); tar = thr._target
                if thId in d: continue
                tarData = [None, None, None] if tar is None else [tar.__name__, inspect.getfile(tar), tar.__doc__]
                d[thId] = [thId, str(thr), False, *tarData, *" "*9]
            ui1 = d.values() | cli.apply(lambda row: [*[html.escape(str(x)) for x in row[:-1]], row[-1]]) | (cli.toJsFunc("term") | cli.grep("${term}") | viz.Table(["thread id", "repr", "isAsync?", "target's name", "target's file name", "target's __doc__", "cron docs", "cron func name", "cron func file name", "running?", "beginTime", "beginTime", "total threadTime (s)", "% utilization", "Stop btn"], height=600, sortF=True)) | cli.op().interface() | cli.toHtml()
            return f"""<div>Total cron utilization: {round(100*totalUtil, 8)}%</div>{ui1}<script>async function {pre}_stop(thId) {{ await fetch(`/k1/thread/api/stop/${{thId}}`); alert("Cron job stopped"); }}</script>"""
        @app.route("/k1/cron", **kwargs)
        def k1_cron_index():
            ui1 = k1.cron.allData().items() | ~cli.apply(lambda k,v: [k, v["delay"], v["delayedStart"], v["lastRun"][1], v["func"].__name__, inspect.getfile(v["func"]), v["docs"]]) | cli.deref() | (cli.toJsFunc("term") | cli.grep("${term}") | viz.Table(["name", "delay", "delayedStart", "lastRun", "func's name", "func's file name", "docs"], height=600, sortF=True)) | cli.op().interface() | cli.toHtml()
            ui2 = k1.cron.allData().items() | ~cli.apply(lambda k,v: [k, f"<h2>{k}</h2><pre>{pprint.pformat(v)}</pre>"]) | cli.deref() | (cli.toJsFunc("term") | cli.grep("${term}", col=0) | cli.cut(1) | cli.join("") | viz.Scroll(600)) | cli.op().interface() | cli.toHtml()
            return f"""<h1>Crons</h1><h2>Overview of all cron jobs</h2><div style="overflow-x: auto">{ui1}</div><h2>Overview of all threads, including cron jobs</h2><div style="overflow-x: auto">{k1_thread()}</div><h2>Detailed cron job's object</h2><div style="overflow-x: auto">{ui2}</div>"""
        @app.route("/k1/thread/api/stop/<int:thId>", **kwargs)
        def k1_thread_api_stop(thId): threading.allData()[thId][2] = False; return "ok"
def scanCron(): # scans all crons and inject in seconds ago, things like that. Only started when cron.flask() is called
    now = time.time()
    if not hasattr(k1lib, "fmt"): return # not ready yet
    for key in _k1_cron_delay_data:
        beginTime = _k1_cron_delay_data[key]["lastRun"][0]
        _k1_cron_delay_data[key]["lastRun"] = [beginTime, k1lib.fmt.time(now - beginTime, metric=True) + " ago"]
@patch(threading)
def allData(): return _k1_threadData

In [84]:
#export
_k1_preload_autoInc = [1]
class preload:
    def __init__(self, refreshTime=10, inactiveTime=60, maxsize=128, name=None, docs=None):
        """Wraps a function and pre executes that function every once in a
while if the function is being used. Example::

    @k1.preload(timeout=10)
    def slow_func(x): time.sleep(0.1); return x * 2

    slow_func(3)                 # takes 100ms
    time.sleep(4);  slow_func(3) # takes 2us
    time.sleep(4);  slow_func(3) # takes 2us
    time.sleep(4);  slow_func(3) # takes 2us
    time.sleep(4);  slow_func(3) # takes 2us
    time.sleep(15); slow_func(3) # takes 100ms

Normally, you don't have to use this, use :class:`~k1lib.cache` instead. This is
for those scenarios where you can't afford to have any delays, as :class:`~k1lib.cache`
will sometime take quite a while to run, but most of the time it runs fast.

With this decorator, if you use the function frequent enough (less than ``inactiveTime``),
then you will never experience any kind of delays, no matter how infrequent. This is good
for generating plots that don't change for a long time, but requires fast server response
all the time.

:param refreshTime: how often to refresh the internal cache to make content fresh?
:param inactiveTime: how long before a function's argument considered inactive and will stop refreshing automatically
:param maxsize: maximum size of the internal cache. See also :class:`~k1lib.cache` """
        if not name: name = f"_k1_preload_{_k1_preload_autoInc[0]}"; _k1_preload_autoInc[0] += 1
        self.refreshTime = refreshTime; self.inactiveTime = inactiveTime; self.maxsize = maxsize
        self.cache = cache(maxsize, refreshTime*3, name=f"preload: {name or '(no name)'}", docs=docs); self.name = name; self.docs = docs
    def __call__(self, func):
        cachedFunc = self.cache(func); lastFetches = collections.defaultdict(lambda: [0, None]) # Dict[key -> [lastFetch, og args]]
        @cron(delay=self.refreshTime, name=f'preload: {(self.name or "(no name)")}', docs=self.docs)
        def k1_preload_watchdog():
            """Continuously fetches data to keep data in k1.preload()'s cache fresh"""
            for lastFetch, [args, kwargs] in list(lastFetches.values()):
                if time.time() - lastFetch < self.inactiveTime: cachedFunc(*args, **kwargs, _k1_cache_refresh=True)
        @functools.wraps(func)
        def wrapper(*args, **kwargs): key = hashableAKw(args, kwargs); lastFetches[key][0] = time.time(); lastFetches[key][1] = [args, kwargs]; return cachedFunc(*args, **kwargs)
        return wrapper

In [197]:
@preload(1, 10)
def slow_func(x): time.sleep(0.1); return x*2, time.time()
times = [];     times.append(time.time()); slow_func(3); times.append(time.time())
time.sleep(4);  times.append(time.time()); slow_func(3); times.append(time.time())
time.sleep(4);  times.append(time.time()); slow_func(3); times.append(time.time())
time.sleep(4);  times.append(time.time()); slow_func(3); times.append(time.time())
time.sleep(4);  times.append(time.time()); slow_func(3); times.append(time.time())
time.sleep(15); times.append(time.time()); slow_func(3); times.append(time.time())
a,b,c,d,e,f = times | k1lib.cli.batched(2) | ~k1lib.cli.apply(lambda x,y: y-x) | k1lib.cli.deref()
assert a > 0.1; assert b < 1e-3; assert c < 1e-3; assert d < 1e-3; assert e < 1e-3; assert f > 0.1

In [8]:
#export
class wrapMod:
    def __init__(self, m, moduleName=None):
        """Wraps around a module, and only suggest symbols in __all__ list
defined inside the module. Example::

    from . import randomModule
    randomModule = wrapMod(randomModule)

:param m: the imported module
:param moduleName: optional new module name for elements (their __module__ attr)"""
        self._init(m, moduleName)
    def _init(self, m, moduleName):
        if moduleName is not None:
            for v in m.__dict__.values():
                v.__module__ = moduleName
        self._wrapMod_moduleName = moduleName; self._wrapMod_m = m
        self.__dict__.update(m.__dict__)
        self._wrapMod_extraDirs = []
    def __dir__(self): return self._wrapMod_m.__all__ + self._wrapMod_extraDirs
    def __str__(self): return str(self._wrapMod_m)
    def __getstate__(self): return {"m": self._wrapMod_m, "name": self._wrapMod_moduleName}
    def __setstate__(self, d): self._init(d["m"], d["name"])
    def __repr__(self): return str(self)

In [9]:
#export
def wraps(ogF):
    """Kinda like :meth:`functools.wraps`, but don't update __annotations__."""
    def inner(f):
        f.__doc__ = ogF.__doc__
        f.__name__ = ogF.__name__
        f.__qualname__ = ogF.__qualname__
        f.__module__ = ogF.__module__
        return f
    return inner

In [10]:
#export
def squeeze(_list:Union[list, tuple, Any], hard=False):
    """If list only has 1 element, returns that element, else returns original
list.

:param hard: If True, then if list/tuple, filters out None, and takes the first
    element out even if that list/tuple has more than 1 element"""
    if isinstance(_list, (tuple, list)):
        if hard: return [e for e in _list if e != None and e != ""][0]
        elif len(_list) == 1: return _list[0]
    if hasTorch and isinstance(_list, torch.Tensor): return _list.squeeze()
    return _list
def raiseEx(ex:Exception):
    """Raises a specific exception. May be useful in lambdas"""
    raise ex
def numDigits(num) -> int:
    """Get the number of digits/characters of this number/object"""
    return len(f"{num}")
def limitLines(s:str, limit:int=10) -> str:
    """If input string is too long, truncates it and adds ellipsis"""
    splits = s.split("\n")
    if len(splits) > limit: return "\n".join(splits[:limit]) + "\n....."
    else: return s
def limitChars(s:str, limit:int=50):
    """If input string is too long, truncates to first `limit` characters of the first line"""
    if s is None: return ""
    s = f"{s}".split("\n")[0]
    return s[:limit-3] + "..." if len(s) > limit else s
def showLog(loggerName:str="", level:int=logging.DEBUG):
    """Prints out logs of a particular logger at a particular level"""
    logger = logging.getLogger(loggerName); logger.setLevel(level)
    sh = logging.StreamHandler(); sh.setLevel(logging.DEBUG); logger.addHandler(sh)
def cleanDiv(_list:List[float], total:int) -> List[int]:
    """Cleanly divides :total: into int segments with weights specified by
:_list:. Looks like this: ``(_list / _list.sum()) * total``, but
everything is an ``int``"""
    _list = np.array(_list)
    _list = (_list*total/_list.sum()).astype(int)
    _list[-1] = total - _list[:-1].sum()
    return _list

In [11]:
assert all(cleanDiv([2, 5], 10) == [2, 8])

In [12]:
#export
def beep(seconds=0.3):
    """Plays a beeping sound, may be useful as notification for long-running tasks"""
    try: import IPython; IPython.core.display.display_html(IPython.display.HTML(f"""<script>(new Audio('data:audio/wav;base64,UklGRl9vT19XQVZFZm10IBAAAAABAAEAQB8AAEAfAAABAAgAZGF0YU'+Array(Math.round(3.3333e3*{seconds})).join(123))).play();</script>"""));
    except: os.system("printf '\a'")

In [13]:
#export
def beepOnAvailable(url:str, timeout=5, **kwargs):
    """Tries to connect with a url repeatedly, and if successful, plays
a beep sound"""
    import requests
    try:
        while True:
            time.sleep(1); successful = False
            try:
                if requests.get(url, timeout=timeout, **kwargs).ok:
                    successful = True
            except: pass
            if successful:
                beep(); break
    except KeyboardInterrupt: print("Still not available")

In [14]:
beepOnAvailable("https://google.com")

In [15]:
#export
def dontWrap():
    """Don't wrap horizontally when in a notebook. Normally, if you're
displaying something long, like the output of ``print('a'*1000)`` in a notebook,
it will display it in multiple lines. This may be undesirable, so this solves
that by displaying some HTML with css styles so that the notebook doesn't wrap."""
    try:
        from IPython.core.display import display, HTML
        display(HTML("""<style>
    div.jp-OutputArea-output pre {white-space: pre;}
    div.output_area pre {white-space: pre;}
    div.CodeMirror > div.highlight {overflow-y: auto;}
</style>"""))
    except: pass#print("Can't run dontWrap()")

In [16]:
#export
import asyncio, functools
from threading import Timer as ThreadingTimer
class AsyncTimer: # rename if want to use
    def __init__(self, timeout, callback):
        self._timeout = timeout; self._callback = callback
    async def _job(self):
        await asyncio.sleep(self._timeout); self._callback()
    def start(self): self._task = asyncio.ensure_future(self._job())
    def cancel(self): self._task.cancel()
def debounce(wait, threading=False):
    """Decorator that will postpone a function's execution until after
``wait`` seconds have elapsed since the last time it was invoked. Taken
from `ipywidgets <https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html>`_.
Example::

    import k1lib, time; value = 0

    @k1lib.debounce(0.5, True)
    def f(x): global value; value = x**2

    f(2); time.sleep(0.3); f(3)
    print(value) # prints "0"
    time.sleep(0.7)
    print(value) # prints "9"

:param wait: wait time in seconds
:param threading: if True, use multiple threads, else just use async stuff"""
    Timer = ThreadingTimer if threading else AsyncTimer
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            if timer is not None: timer.cancel()
            timer = Timer(wait, lambda: fn(*args, **kwargs))
            timer.start()
        functools.update_wrapper(debounced, fn); return debounced
    return decorator

In [17]:
import time; value = 0
@debounce(0.5, True)
def f(x): global value; value = x**2
f(2); time.sleep(0.3); f(3); assert value == 0
time.sleep(0.7); assert value == 9

In [18]:
#export
def scaleSvg(svg:str, scale:float=None) -> str:
    """Scales an svg xml string by some amount."""
    if scale is None: scale = k1lib.settings.svgScale
    wS = w = re.findall("width=\"\\d*pt\"", svg)[0]
    hS = w = re.findall("height=\"\\d*pt\"", svg)[0]
    w = int(int(re.findall("\\d+", wS)[0])*scale)
    h = int(int(re.findall("\\d+", hS)[0])*scale)
    svg = re.sub(wS, f'width="{w}pt"', svg)
    svg = re.sub(hS, f'height="{h}pt"', svg)
    return svg

In [4]:
#export
def now():
    """Convenience function for returning a simple time
string, with timezone and whatnot."""
    import datetime; return datetime.datetime.now().astimezone().isoformat()

In [5]:
now()

'2024-09-29T13:21:03.432194+00:00'

In [20]:
#export
def pushNotification(title="Some title", content="Some content", url="https://k1lib.com"):
    """Sends push notification to your device.

Setting things up:
- Download this app: https://play.google.com/store/apps/details?id=net.xdroid.pn
- Set the `settings.pushNotificationKey` key obtained from the app. Example key: `k-967fe9...`
- Alternatively, set the environment variable `k1lib_pushNotificationKey` instead
- Run the function as usual"""
    import requests
    key = k1lib.settings.pushNotificationKey
    requests.get("http://xdroid.net/api/message?" + urllib.parse.urlencode({'k': key, 't': title, 'c': content, 'u': url}))
    print("Pushed!")

In [21]:
pushNotification("Basics.ipynb unit test")

Pushed!


In [23]:
#export
tickCheckpoints = np.array([1, 2, 2.5, 5, 10])/10
def ticks(x:float, y:float, rounding:int=6):
    """Get tick locations in a plot that look reasonable.
Example::

    ticks(-5, 40)     # returns [-10.0, -5.0, 0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0]
    ticks(0.05, 0.07) # returns [0.05, 0.0525, 0.055, 0.0575, 0.06, 0.0625, 0.065, 0.0675, 0.07, 0.0725]
    ticks(-5, 5)      # returns [-6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

So essentially, whenever you try to plot something, you want both the x and y axis
to not have too many lines, and that the tick values snap to a nice number.

Normally you don't have to do this, as matplotlib does this automatically behind
the scenes, but sometimes you need to implement plotting again, in strange
situations, so this function comes in handy

:param x: start of interval
:param y: end of interval
:param rounding: internally, it rounds tick values to this number of
    digits, to fix tiny float overflows that make numbers ugly. So
    you can disable it if you're working with really small numbers"""
    cli = k1lib.cli; ld = math.log10(y-x); scale = ld-ld%1 # log delta
    sel = ((ld%1 - tickCheckpoints)**2).argmin() # selecting the best fit tick checkpoint
    interval = 10**scale*tickCheckpoints[sel] # interval between ticks
    seed = int(y/10**scale)*10**scale # seed tick
    f1 = cli.applySerial(cli.op()-interval) | cli.breakIf(cli.op()<x-interval)
    f2 = cli.applySerial(cli.op()+interval) | cli.breakIf(cli.op()>y+interval)
    # finally, use the seed to expand in both directions to get all the ticks
    return seed | f1 & f2 | cli.joinStreams() | cli.aS(set) | cli.sort(None) | cli.apply(round, ndigits=rounding) | cli.deref() | cli.aS(np.array)

In [24]:
assert all(ticks(-5, 40) == np.array([-10.0, -5.0, 0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0]))
assert all(ticks(0.05, 0.07) == [0.05, 0.0525, 0.055, 0.0575, 0.06, 0.0625, 0.065, 0.0675, 0.07, 0.0725])
assert all(ticks(-5, 5) == [-6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])

In [25]:
#export
try:
    import graphviz
    def digraph():
        """Convenience method for creating a new graphviz Digraph.
Example::

    g = k1lib.graph()
    g("a", "b", "c")
    g # displays arrows from "a" to "b" and "a" to "c"
"""
        return graphviz.Digraph(graph_attr={"rankdir":"TB"})
    def graph():
        """Convenience method for creating a new graphviz Graph. See also: :meth:`digraph`"""
        return graphviz.Graph(graph_attr={"rankdir":"TB"})
except ImportError:
    digraph = graph = lambda: print("Module `graphviz` not found! Please install it first, something like `pip install graphviz`")

In [26]:
#export
def encode(obj:object) -> str:
    """Serialize random objects into bytes, then turn those bytes to
normal strings using base64. Example::

    a = {"b": 3}
    encode(a) # returns "gASVCgAAAAAAAAB9lIwBYpRLA3Mu"
    decode(encode(a)) # returns {"b"}: 3

See also: :meth:`decode`
"""
    return base64.b64encode(dill.dumps(obj)).decode()
def decode(s:str) -> object:
    """Turns a string produced by :meth:`encode` back into a random object."""
    return dill.loads(base64.b64decode(s.encode()))

In [27]:
a = {"b": 3}; assert decode(encode(a)) == a

In [28]:
#export
import hashlib
def hash(msg:str) -> int:
    """A universal hash function. Why not just use the builtin hash function?
Because it actually changes for different interpreter instances, it won't be
good for code that runs on multiple computers, so this is sort of like a
drop-in replacement. Although it does output an integer, don't rely on it having
the same numeric properties as a normal hash function."""
    if not isinstance(msg, str): msg = f"{msg}"
    m = hashlib.sha256(); m.update(f"{msg}".encode()); return int.from_bytes(m.digest(), "big")
hash(34)

61014538568506221588868294043973711669514098560329658103961376235728675065327

In [29]:
#export
import traceback
try:
    import asyncio, threading, time; from collections import deque
    _coroutineQueue = deque() # deque of (idx, coroutine)
    _coroutineAns = dict() # Dict[idx -> coroutine ans]
    _coroutineAutoIdx = 0
    def _coroutineResolvingThread():
        """Thread that resolves futures passed into k1.resolve(), to turn an async function to a completely normal function"""
        loop = asyncio.new_event_loop()
        while True:
            if len(_coroutineQueue) == 0: time.sleep(0.01)
            else:
                idx, coroutine = _coroutineQueue.popleft()
                # important to recover from exceptions
                try: ans = loop.run_until_complete(coroutine); _coroutineAns[idx] = {"type": "success", "ans": ans}
                except Exception as e: _coroutineAns[idx] = {"type": "failure", "e": f"{e}", "tb": traceback.format_exc()}
    _coroutineResolvingThread_started = [False]
    def _start_coroutineResolvingThread():
        if _coroutineResolvingThread_started[0]: return
        _coroutineResolvingThread_started[0] = True; threading.Thread(target=_coroutineResolvingThread, daemon=True).start()
    _resolve_err = None
except Exception as e: _resolve_err = f"{e}"; _resolve_tb = traceback.format_exc()
def resolve(coroutine):
    """Resolves coroutines without having to use await.
Example::

    async def f(x):
        await asyncio.sleep(1) # simulates heavy processing
        return x + 4
    
    k1.resolve(f(5))

This kinda feels just like ``asyncio.run(f(5))``, so why does this exist?
Here's the docstring of that method:

.. code-block:: text

    This function cannot be called when another asyncio event loop is running in the same thread.

    This function always creates a new event loop and closes it at the end. It should be used as a main entry point for asyncio programs, and should ideally only be called once.

This has more limitations that I found annoying to deal with day-to-day. I want
a function that always work, no matter my setup. So, how this function work
internally is that it spins up a new (permanent, daemon) thread, creates a new
event loop in that thread, then whenever a coroutine comes in, it runs it in
that event loop, returns, then pass control back to whatever thread that called
:meth:`resolve`. I'm sure this can still be messed up in some way, but seems
more useful than the builtin method.

This is just meant as a quick and dirty way to force resolving coroutines. Use
this sparingly, as performance won't be as good as a proper async application.
If you find yourself using this way too often, then I'd suggest reviewing how
:mod:`asyncio` works"""
    global _coroutineAutoIdx; _start_coroutineResolvingThread()
    if _resolve_err: raise Exception(f"k1lib.resolve() not available, encoutered this error while starting up: {_resolve_err}. Traceback:\n{_resolve_tb}")
    idx = _coroutineAutoIdx; _coroutineAutoIdx += 1; _coroutineQueue.append([idx, coroutine])
    while idx not in _coroutineAns: time.sleep(0.01)
    ans = _coroutineAns[idx]; del _coroutineAns[idx]
    if ans["type"] == "success": return ans["ans"]
    else: raise Exception(f"Exception occured while trying to k1lib.resolve(): {ans['e']}. Traceback:\n{ans['tb']}")

In [30]:
async def f(x): return x+4
assert resolve(f(3)) == 7
try: resolve(3)
except: pass
assert resolve(f(4)) == 8

In [31]:
#export
def config(s:str):
    """Convenience method to grab JSON config file from config.mlexps.com"""
    return json.loads("".join(k1lib.cli.cat(f"https://config.mlexps.com/{s}")))

In [16]:
#export
def modbusCrc(msg:list[int]) -> list[int]:
    """Calculates Modbus CRC-16 checksum for a given message.
Example::

    k1.modbusCrc([0x01, 0x03, 0x00, 0x1e, 0x00, 0x01]) # returns (228, 12), or (0xe4, 0x0c)
"""
    crc = 0xFFFF
    for byte in msg:
        crc ^= byte
        for _ in range(8):
            if crc & 0x0001: crc = (crc >> 1) ^ 0xA001
            else: crc >>= 1
    return crc % 256, crc // 256

In [18]:
assert modbusCrc([0x01, 0x03, 0x00, 0x1e, 0x00, 0x01]) == (228, 12)
assert modbusCrc([0x01, 0x03, 0x00, 0x1e, 0x00, 0x01]) == (0xe4, 0x0c)

In [54]:
#export
def parseTimeStr(timeStr):
    """Parses time string and returns 2 unix timestamps.
Example::

    now = time.time()
    k1.parseTimeStr("3 minute") # returns [now - 3*60, now]
    k1.parseTimeStr("3 min")    # returns [now - 3*60, now]
    k1.parseTimeStr("1 hour")   # returns [now - 3600, now]
    k1.parseTimeStr("2.3 hour") # returns [now - 3600*2.3, now]

You get the idea. This is useful in quickly parsing time series graphs between a
time period in the past and now. There are a bunch of period strings with their
corresponding multiplier::

    1,             second # second, sec
    60,            minute # minute, min
    3600,          hour   # hour,   hr
    86400,         day    # day
    7   *  86400,  week   # week,   wk
    30*    86400,  month  # month,  mo
    365.25*86400,  year   # year,   yr
    365.25*864000, decade # decade, dec

Fastest to access are the shortest period strings, like "sec", "min", etc. Use those if
you care about speed.

Unfortunately, this function can't parse composite time strings, like "1 hr 5 min"

:param timeStr: time string to parse"""
    now = time.time()
    if timeStr.endswith("sec"):    return now-float(timeStr[:-3]), now
    if timeStr.endswith("min"):    return now-60 *float(timeStr[:-3]), now
    if timeStr.endswith("hr"):     return now-3600 *float(timeStr[:-2]), now
    if timeStr.endswith("day"):    return now-86400*float(timeStr[:-3]), now
    if timeStr.endswith("wk"):     return now-86400*7*float(timeStr[:-2]), now
    if timeStr.endswith("mo"):     return now-86400*30*float(timeStr[:-2]), now
    if timeStr.endswith("yr"):     return now-86400*365.25*float(timeStr[:-2]), now
    if timeStr.endswith("dec"):    return now-864000*365.25*float(timeStr[:-3]), now

    if timeStr.endswith("second"): return now-float(timeStr[:-6]), now
    if timeStr.endswith("minute"): return now-60 *float(timeStr[:-6]), now
    if timeStr.endswith("hour"):   return now-3600 *float(timeStr[:-4]), now
    if timeStr.endswith("week"):   return now-86400*7*float(timeStr[:-4]), now
    if timeStr.endswith("month"):  return now-86400*30*float(timeStr[:-5]), now
    if timeStr.endswith("year"):   return now-86400*365.25*float(timeStr[:-4]), now
    if timeStr.endswith("decade"): return now-864000*365.25*float(timeStr[:-6]), now
    if timeStr == "lastHour": return now-3600, now # "last" + period is sort of a hidden mode, to ensure backwards compatible with other systems I have, but let's not expose it in the docs
    if timeStr == "lastDay": return now-86400, now
    if timeStr == "lastWeek": return now-86400*7, now
    raise flask.FailureException(f"Don't understand '{timeStr}'. Has to be int followed by either 'hour' or 'day' or 'week'")

In [53]:
x,y = parseTimeStr("2.3 hr"); assert y-x == 8280

In [89]:
!../export.py _basics --upload=True

./export started up - /home/quang/miniforge3/bin/python
----- exportAll
16791   0   60%   
11083   1   40%   
installing...
Found existing installation: k1lib 1.8
Uninstalling k1lib-1.8:
  Successfully uninstalled k1lib-1.8
DEPRECATION: Loading egg at /home/quang/miniforge3/lib/python3.12/site-packages/aigu-0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, http://10.104.0.3:3141/
Processing /home/quang/k1lib
  Preparing metadata (setup.py) ... done
  Created wheel for k1lib: filename=k1lib-1.8-py3-none-any.whl size=5135060 sha256=8dfc67b527d2e358d332ba2b5d167e09aa4d535808412db47636034ce78e547e
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ze1yopz/wheels/b5/32/67/e20c84dce16d707fb881c12d405f70adfaa36fe7dae9021380
Successfully built k1lib
installed
uploading...
uploaded


In [80]:
!../export.py _basics

./export started up - /home/quang/miniforge3/bin/python
----- exportAll
16688   0   60%   
11040   1   40%   
installing...
Found existing installation: k1lib 1.8
Uninstalling k1lib-1.8:
  Successfully uninstalled k1lib-1.8
DEPRECATION: Loading egg at /home/quang/miniforge3/lib/python3.12/site-packages/aigu-0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, http://10.104.0.3:3141/
Processing /home/quang/k1lib
  Preparing metadata (setup.py) ... done
  Created wheel for k1lib: filename=k1lib-1.8-py3-none-any.whl size=5131893 sha256=dd5782cf6dac456b73842017fd3164470ba869e079b1c94d033dda08f64a3cb0
  Stored in directory: /tmp/pip-ephem-wheel-cache-nw0am__r/wheels/b5/32/67/e20c84dce16d707fb881c12d405f70adfaa36fe7dae9021380
Successfully built k1lib
installed


In [282]:
!../export.py _basics --bootstrap=True

./export started up - /home/quang/miniforge3/bin/python
----- bootstrapping
Current dir: /home/quang/k1lib, /home/quang/k1lib/k1lib/../export.py
Found existing installation: k1lib 1.7.3
Uninstalling k1lib-1.7.3:
  Successfully uninstalled k1lib-1.7.3
DEPRECATION: Loading egg at /home/quang/miniforge3/lib/python3.12/site-packages/aigu-0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, http://10.104.0.3:3141/
Processing /home/quang/k1lib
  Preparing metadata (setup.py) ... done
  Created wheel for k1lib: filename=k1lib-1.7.3-py3-none-any.whl size=5111617 sha256=0fee298aa849b983be8ca39d5855f76293c245a8760f36417081c0eb99afe5ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-ewuoj141/wheels/b5/32/67/e20c84dce16d707fb881c12d405f70adfaa36fe7dae9021380
Successfully built k1lib
